## Constants

In [ ]:
import math
import scipy.constants as constant

# Carrier Frequency 
FC = 2e9  

# Speed of Light
C = constant.c  

# Power Spectral Density
N0 = 1e-20  

# Bandwidth
B = 1e6  

# Noise
STD = B * N0  

# Environmental Variable 1
B1 = 0.36  

# Environmental Variable 2
B2 = 0.21  

# Path Loss Exponent
ALPHA = 2   

# Additional path loss for LineOfSight
U_LOS = 10**(3/10)  

# Additional path loss for NonLineOfSight
U_NLOS = 10**(23/10)  

K0 = (4 * math.pi * FC / C) ** 2

# Environmental Variable 3
ZETA = 0  

# Transmission power of an UAV
P = 0.08  

# Discount Rate
GAMMA = 0.95  

# Learning Rate
LR = 0.001  

# Initial Epsilon Greedy Value
EPSILON = 1  

# Epsilon Greedy Decay Rate
EPSILON_DECAY = 0.95  

# Minimum Epsilon Greedy
EPSILON_MIN = 0.1  

# Minimum altitude to fly
MIN_ALTITUDE = 200

# Initial Altitude
ALTITUDE = 400  

# Maximum altitude to fly
MAX_ALTITUDE = 800  

# Penalty value
PENALTY = -100   

# UAV speed rate according to humans
UAV_MOVEMENT_SCALE= 20

TEST_ENV_FILE = "User_Data/test_env.pkl"

In [ ]:
import random as rand 
import tensorflow as tf
from time import sleep
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from collections import deque 
import time
class Agent(): 
    def __init__(self,state_size,action_count,batch_size=200,maxlen=10000):
        self.batch_size = batch_size
        self.input_size   = state_size 
        self.action_count = action_count 
        self.lr = LR
        self.gamma = GAMMA
        self.epsilon = EPSILON
        self.epsilon_decay = EPSILON_DECAY
        self.epsilon_min = EPSILON_MIN
        self.model = self.build_model() # Deep Q Network
        self.memory = deque(maxlen=maxlen) # Experience Memory

    def build_model(self):
        """
            Create Neural network as Deep Q Network
        """    
        model = Sequential()
        model.add(Dense(128, input_dim= self.input_size, activation = "relu"))   
        model.add(Dense(64,activation="relu"))   
        model.add(Dense(64,activation="relu"))   
        model.add(Dense(64,activation="relu"))  
        model.add(Dense(64,activation="relu"))  
        model.add(Dense(self.action_count,activation="linear"))
        model.compile(Adam(learning_rate = self.lr ,clipnorm=1.0),loss=Huber())
        return model  

    def store(self, state, action, reward, next_state, done):
        """
            Store experience in memory
        """
        self.memory.append((state, action, reward, next_state, done))

    def act(self,state,cannot_random=False):
        """
            Choosing action either random or using Deep Q Network
        """
        if (rand.uniform(0,1) <= self.epsilon and not cannot_random):
            return rand.randint(0,self.action_count-1) 
        else:
            act_values = self.model.predict(state) 
            return np.argmax(act_values[0])
        
    def is_memory_enough(self):
        """
            Check if memory is full enough for a batch
        """
        return not (len(self.memory)< self.batch_size)
    
    def replay(self):
        """
            Sample random experiences from memory and replay them
        """
        if(not self.is_memory_enough()):
            return
        
        minibatch = rand.sample(self.memory,self.batch_size)
        
        for state,action, reward,next_state, done in minibatch:  
            
          if done:
            target = reward
          else:  
            target = reward + self.gamma*np.amax(self.model.predict(next_state))  
           
          train_target = self.model.predict(state)
          train_target[0][action] = target 
          
          self.model.fit(state,train_target,verbose=0,workers=8,use_multiprocessing=True)

    def adaptiveEGreedy(self):
        """
            Decay epsilon
        """
        if(self.epsilon > self.epsilon_min and self.is_memory_enough()):
            self.epsilon *= self.epsilon_decay   
        print("current_randomness: ", self.epsilon)

from scipy.stats import truncnorm

# Helper Function for normal distribution
def get_truncated_normal(mean=0, low=0, upp=10):
    return truncnorm(
        (low - mean), (upp - mean), loc=mean, scale=1).rvs()
    
# Normal Distribution
def gaussian(end,begin=0):
    normal_random = round(get_truncated_normal(mean=int((begin+end)/2),low=begin,upp=end))
    return max(0,min(normal_random,end))


# General Class for moving objects to extend from.
class MovingObject():
    def __init__(self,initial_location):
        self.current_location = initial_location.copy()
        self.starting_location = initial_location.copy()

    def reset(self):
        self.current_location = self.starting_location.copy()


#UAV class
class UAV(Agent,MovingObject): 
    def __init__(self,state_size,action_count,batch_size,initial_location,maxlen=10000):
        Agent.__init__(self,state_size,action_count,batch_size=batch_size,maxlen=maxlen)
        MovingObject.__init__(self,initial_location)
        self.transmission_power = P


#User class
class UE(MovingObject):
  def __init__(self,initial_location,env_dim,movement_vectors,define_path_first = True,step_count = 10000,movement_function=gaussian):
    super().__init__(initial_location)

    self.movement_vectors =movement_vectors[:5].copy()
    rand.shuffle(self.movement_vectors)

    self.movement_function = movement_function
    self.maxY , self.maxX = env_dim
    self.path = []
    self.path_determined = define_path_first

    if define_path_first:
      self.initial = 0
      for _ in range(step_count):
        self.path.append(movement_function(len(self.movement_vectors)-1))
    
  def move(self):
    action_index = 0
    if self.path_determined:
      action_index = self.path[self.initial]
      self.initial+=1
      if self.initial==len(self.path):
        self.initial = 0
    else:
      action_index = self.movement_function(len(self.movement_vectors)-1)

    action = self.movement_vectors[int(action_index)]
    self.current_location= [self.current_location[i] + action[i] for i in range(3)]

# Environment Class

In [ ]:
from statistics import mean
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import time
from keras.models import load_model
import pickle

sns.set_style("darkgrid")


class UavSimuEnv():
  actions = [(1,0,0),(-1,0,0),(0,0,0),(0,1,0),(0,-1,0)]  # Actions
  coord_count = 2
      
  def distance_func(self,v1,v2): 
    """ x and y is unit, altitude is meter so calculated distance this way """
    return (sum(((p-q)*UNIT)**2 for p, q in zip(v1[:2], v2[:2])) + (v1[2]-v2[2])**2)** .5

  def get_all_uavs(self):
        return [uav for uav in self.map["uav_set"]]
        
  def save(self,env_name): 
    """
      save models and memories of all UAVs
    """
    for index,uav in enumerate(self.get_all_uavs()):
        model = uav.model
        path = "modelss/{}/".format(env_name)
        if not os.path.exists(path):
            os.makedirs(path)
        model_name = "{}uav{}".format(path,index)
        model.save(model_name+".h5")
        pickle.dump(uav.memory, open(model_name+"memory.pkl", 'wb'))
            
            
  def load(self,env_name,just_for_test=True):
    """
      load models and memories for all UAVs
    """
    for index in range(len(self.map["uav_set"])):
        path = "modelss/{}/".format(env_name)
        model_name = "{}uav{}".format(path,index)
        self.getUAV(index).model = keras.models.load_model(model_name+".h5",compile=just_for_test)
        self.getUAV(index).memory = pickle.load(open(model_name+"memory.pkl", "rb"))

            
  def get_input_size(self,uav_count):
    return uav_count*self.coord_count + math.ceil(self.step_count/UAV_MOVEMENT_SCALE)

  def is_collect_step(self):
    return self.getUAV(0).is_memory_enough() 
               
  def get_current_epsilon(self):
    return self.getUAV(0).epsilon
    
  def __init__(self,uav_paths,ue_set,env_dim = (100,100),batch_size=200,max_memory_per_agent=10000,uav_class=UAV):
    self.step_count = STEP_COUNT
    self.uav_count = len(uav_paths) 
    self.env_dim = env_dim 
    self.input_size = self.get_input_size(len(uav_paths))
    self.map = {"uav_set": self.init_uav(uav_paths,batch_size,max_memory_per_agent,uav_class), "ue_set": ue_set}

  def init_uav(self,uav_paths,batch_size,max_memory_per_agent,uav_class):
    """
      Initialize Agents
    """
    print(uav_class)
    return [uav_class(self.input_size, len(self.actions), batch_size, begin,maxlen=max_memory_per_agent) 
            for begin in uav_paths]


  def get_distance_list(self, dest_list, location,isObject=False):
    """
      Return distance list of distance between the locations in dest;_list and given location
    """
    if (isObject):
      return [self.distance_func(location,loc.current_location) for loc in dest_list]
    return [self.distance_func(location,loc) for loc in dest_list] 
    
  def reset(self):
    """
      Reset Environment
    """
    def reset_set(key):
      for index, val in enumerate(self.map[key]):
        self.map[key][index].reset()
    
    reset_set("ue_set")
    reset_set("uav_set")
    self.initial_time = 0 

  def get_state_input(self,isNext=False):
    """
      Return state information for UAV
    """
    all_uav_coordinates = []

    for uav in self.map["uav_set"]:
      all_uav_coordinates.extend(uav.current_location[:self.coord_count]) 
    
    time_range = [0 for _ in range(math.ceil(self.step_count/UAV_MOVEMENT_SCALE))]
    print(self.initial_time // UAV_MOVEMENT_SCALE)
    print(self.initial_time)
    time_range[self.initial_time // UAV_MOVEMENT_SCALE] = 1
    return self.reshape(all_uav_coordinates +time_range)

    
  def get_distance_matrix(self,map_obj=None):
    """
      Return all distance matrix and association count vector between UAV-UE 
    """
    if(map_obj==None):
        map_obj = self.map
        
    def minIndex(lst):
      return lst.index(min(lst))    

    def get_distances(lst1, lst2, isLst2Object = True):
      distance_matrix = []
      assoc_matrix = []
      for _ in range(len(lst2)):
        assoc_matrix.append([])

      for index,member in enumerate(lst1):
        distances = self.get_distance_list(lst2,member.current_location,isLst2Object)
        distance_matrix.append(distances)
        index_of_min = minIndex(distances)
        assoc_matrix[index_of_min].append(index)

      return distance_matrix, assoc_matrix

    ue_uav_matrix, assoc_matrix_uav = get_distances(map_obj["ue_set"],map_obj["uav_set"]) 

    return {"ue_uav_matrix": ue_uav_matrix, "assoc_matrix_uav" : assoc_matrix_uav}


  def calculate_sum_rate(self,map_obj=None):
    
    if(map_obj==None):
        map_obj = self.map
        
    uav_count = len(map_obj["uav_set"])
    ue_count = len(map_obj["ue_set"])
    distance_state_dict = self.get_distance_matrix(map_obj)
    sumrate = 0
    transmit_powers = []
    channel_gain_matrix = []
    
    # Nested Function
    def calculate_transmit_power(uav_index):
      """
        calculate transmit power of uav
      """
      uav = map_obj["uav_set"][uav_index]
      uav_assoc_count = len(distance_state_dict["assoc_matrix_uav"][uav_index])
    
      #Eğer uav hiçbir servis yapmıyorsa transmit gücünü kullanmayacağı için 0 olacak
      if(uav_assoc_count==0):
            return 0 
      p = uav.transmission_power / uav_assoc_count

      return p

    # Nested Function
    def calculate_channel_gain(uav_index,user_index):
      """
        calculate channel gain between uav and user
      """
      uav = map_obj["uav_set"][uav_index]
      d = distance_state_dict["ue_uav_matrix"][user_index][uav_index]
      theta = math.asin(uav.current_location[-1] / d )
      Plos = B1 * (180 * theta / math.pi - ZETA ) ** B2
      
      Pnlos = 1 - Plos
      g = (K0 ** (-1)) * (d ** (- ALPHA)) * ((Plos*U_LOS + Pnlos*U_NLOS) **(-1))
      return g


    #First calculate all channel gains and transmit powers of all combination of uav and users
    for uav_index in range(uav_count):

      p = calculate_transmit_power(uav_index)
      transmit_powers.append(p)
      channel_gain_list = [calculate_channel_gain(uav_index,user_index) \
                           for user_index in range(ue_count)]
      channel_gain_matrix.append(channel_gain_list)
    
    # Nested Function
    def calculate_interference(uav_index,user_index):
      """
        Calculate interference between uav and user caused by other uavs
      """
      I = 0

      for other_uav_index in range(uav_count):
        if (other_uav_index==uav_index):
          continue
        p = transmit_powers[other_uav_index]
        g = channel_gain_matrix[other_uav_index][user_index]
        I +=  p*g 
        
      return I
    

    for uav_index in range(uav_count):

        p = transmit_powers[uav_index]
        users_of_uav = distance_state_dict["assoc_matrix_uav"][uav_index]
        for user_index in users_of_uav:
          
          I = calculate_interference(uav_index,user_index)
          g = channel_gain_matrix[uav_index][user_index]
          SINR = p*g/(I + STD)
          sumrateOfUser = B * math.log2(1+SINR)
          sumrate += sumrateOfUser 
            
    return sumrate*1e-7

  def step_UEs(self):
    """
      Move all UEs in Simulation
    """
    ue_set_length = len(self.map["ue_set"])
    for ue in self.map["ue_set"]:
      ue.move()

      if (not self.isInside(ue.current_location,True)):
        self.map["ue_set"].remove(ue)


  def isCollides(self,uav_index,new_location): 
    """
      check if location is against limits
    """
    for index,uav in enumerate(self.map["uav_set"]): 
        if(index!=uav_index and np.array_equal(uav.current_location, new_location)):
            return True
    return False


  def step_env(self,action_indexes):
        """
          Change environment with given joint actions
        """
        old_sum_rate = self.calculate_sum_rate()
        penalty = 0
        done = False
        for uav_index in range(len(self.map["uav_set"])):
            current = self.getUAV(uav_index).current_location
            action_index = action_indexes[uav_index]
            new_location = [current[i] + self.actions[action_index][i]
                                                         for i in range(3)]
            if(not self.isInside(new_location)):
                penalty += PENALTY
                done = True
            else:
                self.getUAV(uav_index).current_location = new_location
        
        for uav_index in range(len(self.map["uav_set"])):
            if(self.isCollides(uav_index,self.getUAV(uav_index).current_location)):
                done = True
                penalty +=PENALTY
                
        new_sum_rate = self.calculate_sum_rate()
        
        reward = new_sum_rate-old_sum_rate + penalty #penalty negatif
        done = done or self.initial_time == self.step_count
        return self.get_state_input(), reward, done
    
     
  def getUAV(self,uav_index):
    return self.map["uav_set"][uav_index]


  def step_UAVs(self,isTest=False,isCollectStep=False):
    """
      Move all UAVs
    """
    uav_set_length = len(self.map["uav_set"])
    
    state = self.get_state_input()
  
    action_indexes = [self.getUAV(uav_index).act(state,cannot_random = isTest) \
               for uav_index in range(uav_set_length)]
        
    next_state, reward, Done = self.step_env(action_indexes)
  
    if(not isTest):
        for uav_index in range(uav_set_length):
            self.getUAV(uav_index).store(state,action_indexes[uav_index],reward,next_state,Done) 
            
            if(not isCollectStep):
                self.getUAV(uav_index).replay()
                if(self.initial_time==self.step_count-1):
                    self.getUAV(uav_index).adaptiveEGreedy() 
    self.initial_time +=1
        
            
            
  def reshape(self, data):
    return np.reshape(data, [1,self.input_size]) 


  def isInside(self,location,isOnGround=False):
    """
      Return if location is against the limits
    """
    return (0 <= location[0] < self.env_dim[1] and 0<= location[1] < self.env_dim[0] ) \
            and (isOnGround or  MIN_ALTITUDE <= location[2] <= MAX_ALTITUDE)

  def get3DMap(self):
      def getCoords(map):
        x,y,z = [],[],[]
        for obj in map:
          x.append(obj.current_location[0])
          y.append(obj.current_location[1])
          z.append(obj.current_location[2])
        return x,y,z
      return getCoords(self.map["uav_set"]),getCoords(self.map["ue_set"])

    
  def render(self):

    """
      Render simulation visually
    """
    fig = plt.figure(figsize=(10,10))
    ax = Axes3D(fig)
    uav_coords,ue_coords = self.get3DMap()
    clear_output(wait=True)
    ax.set_xlim3d(0,self.env_dim[0])
    ax.set_ylim3d(0,self.env_dim[1])
    ax.set_zlim3d(0,MAX_ALTITUDE)
    ax.scatter(*uav_coords,c="green")
    ax.scatter(*ue_coords,c="red")
    distance_map = self.get_distance_matrix()["assoc_matrix_uav"]
    for uav_index in range(self.uav_count):
        coord_of_UAV = self.getUAV(uav_index).current_location
        for ue_index in distance_map[uav_index]: 
            coord_of_UE = self.map["ue_set"][ue_index].current_location
            ax.plot([coord_of_UE[0],coord_of_UAV[0]],[coord_of_UE[1],coord_of_UAV[1]],[coord_of_UE[2],coord_of_UAV[2]],c="green",alpha=.1)
    plt.show()

# Simulation Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from keras.optimizers import Adam 
import multiprocessing
from joblib import Parallel, delayed

def plot_iter_graph(ignore_count,sum_rates_per_iteration):
    plt.plot(sum_rates_per_iteration[ignore_count:])
    plt.title("Sum rates per iteration")
    plt.show(sum_rates_per_iteration)

def plot_step_graph(iteration_num, step_num,step_count,sum_rates_per_step):
  if (iteration_num >= 1): 
    if (step_num >1):
      begin_iter = max(0,iteration_num-5)
      x_pivot = begin_iter*step_count
      print(begin_iter,x_pivot,iteration_num)
      show_range = range(x_pivot , len(sum_rates_per_step))
      plt.plot(show_range,sum_rates_per_step[x_pivot:])
      plt.title("Sum rates per step") 
      for t in range(5):
        plt.axvline(x=(t+begin_iter)*step_count,color="red")
    
      plt.show()

# Train environment
def simulate(env,iteration_count = 50, step_count = 20,batch_size=200,consider_terminal_state=False,collect_step_size=100,
            env_name=None,onlyForTest=False):

  sum_rates_per_iteration = []
  sum_rates_per_step = []
  max_score = 0
  max_other_score = 0
  for iter_num in range(iteration_count + (collect_step_size//step_count)):
    
    
    counter = 0  
    isCollectStep = collect_step_size > (iter_num*step_count)
    env.test_mode = False
    env.reset() 
    while (not onlyForTest and counter!=step_count and (isCollectStep == (collect_step_size > (iter_num*step_count + counter) ))):
        
        
        """if counter%UAV_MOVEMENT_SCALE==0:
          env.step_UEs()"""
        
        #Step simulation
        env.step_UAVs(isCollectStep=isCollectStep) 
        counter += 1
        
        #Plot sum rates by step and interation
        clear_output(wait=False)
 
        plot_iter_graph(0,sum_rates_per_iteration)
        plot_step_graph(iter_num,counter,step_count,sum_rates_per_step)
        
        print(env.get_current_epsilon())
        print("Training, step: ",counter)
        print("Iteration: ",iter_num)
         

    #TESTING
    counter = 0
    total_sum_rate = 0
    env.test_mode = True
    env.reset()
    while (counter!=step_count and not isCollectStep):

        """if counter%UAV_MOVEMENT_SCALE==0:
          env.step_UEs()"""
        #Step UAVs
        env.step_UAVs(isTest=True)

        #Get Error
        sum_rate = env.calculate_sum_rate()
        sum_rates_per_step.append(sum_rate)
        total_sum_rate += sum_rate
        counter+=1
        #Step UEs 
        env.render() 
        

        #Plot resource errors by step and interation
        plot_step_graph(iter_num,counter,step_count,sum_rates_per_step)
        
        print("Testing...")
        print("Counter: ",counter)
        print("Iteration Number: ",iter_num)
        
    print(total_sum_rate/step_count)
    if(not isCollectStep):
        score = total_sum_rate/step_count
        other_score = sum_rates_per_step[-1]
        sum_rates_per_iteration.append(score)
        if(score > max_score):
            env.save(env_name)
            max_score = score 
        if(other_score > max_other_score):
            env.save(env_name+"_bestend")
            max_other_score = other_score 
  return sum_rates_per_iteration, sum_rates_per_step

# Extended Environment and Agent Classes

In [ ]:
class ThreeDOFUavSimu(UavSimuEnv):
    coord_count = 3
    actions = [(1,0,0),(-1,0,0),(0,0,0),(0,1,0),(0,-1,0),(0,0,-1),(0,0,1)] #Actionlar

In [ ]:
class IndependentAISimuEnv(UavSimuEnv):
    
    def step(self,uav_index,action_index):
        
        #Get current and new locations of uav
        current_loc = self.getUAV(uav_index).current_location
        new_location = [current_loc[i] + self.actions[action_index][i]
                                                         for i in range(3)]
        
        #Calculate Reward
        reward = 0
        done = False
        if(not self.isInside(new_location) or self.isCollides(uav_index,new_location)):
            reward = PENALTY
            done = True
        else:
            old_sum = self.calculate_sum_rate()
            self.getUAV(uav_index).current_location = new_location
            new_sum = self.calculate_sum_rate()
            reward = new_sum - old_sum
        
        #If uav is the last one then increase step
        if(uav_index==self.uav_count-1):
            self.initial_time+=1
            
        #Get next state (Next uav's next input = next observation of next agent)
        next_obs = self.get_state_input(uav_index)
        done = done or self.initial_time == self.step_count
        return reward,next_obs,done 
    
    def train_agents(self):
        for uav_index in range(self.uav_count):
            self.getUAV(uav_index).replay() 
            print("\nMemory: \n ",self.getUAV(uav_index).memory[-1])
            print("\nEpsilon: ",self.getUAV(uav_index).epsilon,"\n")
            if(self.initial_time==self.step_count-1):
                self.getUAV(uav_index).adaptiveEGreedy() 
                    
    def step_UAVs(self,save_reward=True,isTest=False,isCollectStep=False):  
        
        #For each uav, get (observation,action,reward,next_state)
        for uav_index in range(self.uav_count):
            
            observation = self.get_state_input(uav_index) 
            
            #Get action index of uav
            action = self.getUAV(uav_index).act(observation, cannot_random=isTest)
            
            #step env
            reward,next_obs,done = self.step(uav_index,action)
            
            #If training, agent stores
            if(not isTest):
                self.getUAV(uav_index).store(observation,action,reward,next_obs,done) 
        
        #After a step finishes, agent replays
        if(not isTest and not isCollectStep):
            self.train_agents()

In [ ]:
from keras import backend as K
from keras.losses import Huber
import tensorflow as tf

class DDQNAgent(UAV):
    
    def update_target_model(self):
        # copy weights from model to target_model
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i]
        self.target_model.set_weights(target_weights)
        
    def __init__(self,*args,**kwargs): 
        super().__init__(*args,**kwargs)
        self.target_model = tf.keras.models.clone_model(self.model)
        

class DDQNAgentDiff(DDQNAgent): 
    
    def replay_with_indexes(self,indexes,next_agent_target):
        minibatch = [self.memory[index] for index in indexes]
        self.replay_on_batch(minibatch,next_agent_target)
    
    def replay_on_batch(self,minibatch,next_agent_target):
        if(not self.is_memory_enough() ):
            return
        for state,action, reward,next_state, done in minibatch:  
            
            if done:
                target = reward
            else:  
                target = reward + self.gamma*np.amax(next_agent_target.predict(next_state))   
            train_target = self.model.predict(state)
            train_target[0][action] = target
 
            self.model.fit(state,train_target,verbose=1,use_multiprocessing=True)
        self.update_target_model()
    
    def replay(self,next_agent_target):
        
        if(not self.is_memory_enough() ):
            return
        minibatch = rand.sample(self.memory,self.batch_size)
        self.replay_on_batch(minibatch,next_agent_target)
        

In [ ]:
from keras.losses import Huber
from keras.optimizers import RMSprop
    
class UavSimuEnvWithComm(UavSimuEnv):
               
    def is_collect_step(self):
        return self.getUAV(0).is_memory_enough() 
               
    def get_current_epsilon(self):
        return self.getUAV(0).epsilon
    
    def train_agents(self):
        for uav_index in range(self.uav_count):
            next_uav_index = (uav_index+1)%(self.uav_count)
            next_uav_policy = self.getUAV(next_uav_index).model
            self.getUAV(uav_index).replay(next_uav_policy) 

            print("\nMemory: \n ",self.getUAV(uav_index).memory[-1])
            print("\nEpsilon: ",self.getUAV(uav_index).epsilon,"\n")

            if(self.initial_time==self.step_count-1):
                self.getUAV(uav_index).adaptiveEGreedy() 
    
    def isCollides(self,uav_index,new_location): 
        for index,uav in enumerate(self.map["uav_set"]): 
            if(index>uav_index):
                return False
            elif(index!=uav_index and np.array_equal(uav.current_location, new_location)):
                return True
        return False

    def get_state_input(self,uav_index):
        all_uav_coordinates = []

        all_uav_coordinates.extend(self.getUAV(uav_index).current_location[:self.coord_count])
        for index,uav in enumerate(self.map["uav_set"]):
            if(uav_index==index):
                continue 
            all_uav_coordinates.extend(uav.current_location[:self.coord_count]) 
            
        time_range = [0 for _ in range(math.ceil(self.step_count/UAV_MOVEMENT_SCALE))]
        time_range[self.initial_time // UAV_MOVEMENT_SCALE] = 1
         
        return self.reshape(all_uav_coordinates +time_range)

    def step(self,uav_index,action_index):
        
        #Get current and new locations of uav
        current_loc = self.getUAV(uav_index).current_location
        new_location = [current_loc[i] + self.actions[action_index][i]
                                                         for i in range(3)]
        
        #Calculate Reward
        reward = 0
        done = False
        if(not self.isInside(new_location) or self.isCollides(uav_index,new_location)):
            reward = PENALTY
            done = True
        else:
            old_sum = self.calculate_sum_rate()
            self.getUAV(uav_index).current_location = new_location
            new_sum = self.calculate_sum_rate()
            reward = new_sum - old_sum
        
        #Get next uav's index
        next_uav_index = (uav_index+1)%(self.uav_count)
        
        #If uav is the last one then increase step
        if(uav_index==self.uav_count-1):
            self.initial_time+=1
            
        #Get next state (Next uav's next input = next observation of next agent)
        next_obs = self.get_state_input(next_uav_index)
        done = done or self.initial_time == self.step_count
        return reward,next_obs,done 

    def step_UAVs(self,save_reward=True,isTest=False,isCollectStep=False):  
        
        
        next_obs = self.get_state_input(0)
        #For each uav, get (observation,action,reward,next_state)
        for uav_index in range(self.uav_count):
            
            observation = next_obs.copy()
            
            #Get action index of uav
            action = self.getUAV(uav_index).act(observation, cannot_random=isTest)
            
            #step env
            reward,next_obs,done = self.step(uav_index,action)
            
            #If training, agent stores
            if(not isTest):
                self.getUAV(uav_index).store(observation,action,reward,next_obs,done) 
        
        #After a step finishes, agent replays
        if(not isTest and not isCollectStep):
            self.train_agents()
            
    def __init__(self,uav_paths,ue_set,env_dim = (100,100),batch_size=200,max_memory_per_agent=10000,uav_class=UAV):
        super().__init__(uav_paths,ue_set,env_dim = (100,100),batch_size=batch_size,max_memory_per_agent=max_memory_per_agent,uav_class=uav_class)
        self.map["uav_set"] = self.init_uav(uav_paths,batch_size,max_memory_per_agent,uav_class)
        

class UavSimuEnvWithCommSameReplayIndexesReversedReplay(UavSimuEnvWithComm): 
        
    def train_agents(self):
        indexes = rand.sample(range(len(self.getUAV(0).memory)), self.getUAV(0).batch_size)
        for uav_index in reversed(range(self.uav_count)):
            
            next_uav_index = (uav_index+1)%(self.uav_count)
            next_uav_policy = self.getUAV(next_uav_index).model
            self.getUAV(uav_index).replay_with_indexes(indexes,next_uav_policy) 
            
            print("\nMemory: \n ",self.getUAV(uav_index).memory[-1])
            print("\nEpsilon: ",self.getUAV(uav_index).epsilon,"\n")
            
            if(self.initial_time==self.step_count-1):
                self.getUAV(uav_index).adaptiveEGreedy() 


In [ ]:
from keras.layers import AveragePooling2D,Flatten, Conv2D, Input, concatenate
from random import randint
from keras import Model
class semiConvAgent(DDQNAgent):
    
    def build_model(self):
        inputA = Input(shape=(self.input_size[0],))
        inputB = Input(shape=list(self.input_size[1])+[1])
        
        uav_coord_input = Dense(64, activation="relu")(inputA)
        uav_coord_input = Model(inputs=inputA, outputs=uav_coord_input)
        
        ue_coord_input = AveragePooling2D(pool_size=(4, 4),padding="same")(inputB)
        ue_coord_input = Flatten()(ue_coord_input)
        ue_coord_input = Model(inputs=inputB, outputs=ue_coord_input)

        combined = concatenate([uav_coord_input.output, ue_coord_input.output])
        
        z = Dense(512, activation="relu")(combined)
        z = Dense(512, activation="relu")(z)
        z = Dense(256, activation="relu")(z)
        z = Dense(self.action_count,activation="linear")(z)
        
        model = Model(inputs=[uav_coord_input.input, ue_coord_input.input], outputs=z)
        model.compile(Adam(learning_rate = self.lr ,clipnorm=1.0),loss=Huber())
        print(model.summary())
        return model  
    
class semiConvAgentCooperative(DDQNAgentDiff,semiConvAgent):
    
    def build_model(self):
        return semiConvAgent.build_model(self)
    
class UserScalableEnv(IndependentAISimuEnv):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        
    def get_input_size(self,uav_count):
        return (uav_count*self.coord_count,self.env_dim)
    
    #Point process Distribution
    def create_random_user_locations(self):
        total_intensity = self.env_dim[0]*self.env_dim[1]*(np.exp(1)-1) / 5
        max_intensity = np.exp(1)/5

        number_points = np.random.poisson(self.env_dim[0] * self.env_dim[1] * max_intensity)
        ue_coords = []
        ue_count = len(self.map["ue_set"])
        
        while(len(ue_coords)<ue_count):
            x = randint(0, self.env_dim[1]-4)
            y = randint(0, self.env_dim[0]-4)
            intensity = np.exp(y / self.env_dim[1]) / 5
            if intensity >= np.random.uniform(0, max_intensity):
                ue_coords.append([x,y,0])
        return ue_coords
    
    
    def reset(self):
        def reset_set(key):
          for index, val in enumerate(self.map[key]):
            self.map[key][index].reset()
            
        ue_set = []
        
        if(self.test_mode==True):
            with open(TEST_ENV_FILE, 'rb') as data: 
                ue_set = pickle.load(data)
        else:
            random_locs = self.create_random_user_locations()
            for coord in random_locs:
                ue_set.append(UE(coord,env_dim,UavSimuEnv.actions,define_path_first=True,step_count=self.step_count//UAV_MOVEMENT_SCALE))
        
        self.map["ue_set"] = ue_set
        reset_set("uav_set")
        self.initial_time = 0 

    def get_state_input(self,uav_index):
        all_uav_coordinates = []

        for uav in self.map["uav_set"]:
          all_uav_coordinates.extend(uav.current_location[:self.coord_count]) 

        user_map = np.zeros((env_dim[0],env_dim[1]))
        for user in self.map["ue_set"]:
            row = user.current_location[0]
            col = user.current_location[1]
            user_map[row][col] = 1
            
        return [self.reshape(np.array(all_uav_coordinates)),self.reshape(user_map)]
                   
    def reshape(self, data):
        return np.reshape(data, (1,) + data.shape) 
    
class UserScalableEnvWithTimeInfo(UserScalableEnv):
    
    def reshape(self, data):
        return np.reshape(data, (1,) + data.shape) 
    
    def get_input_size(self,uav_count):
        return (uav_count*self.coord_count+1,self.env_dim)
    
    def get_state_input(self,uav_index):
        all_uav_coordinates = []

        for uav in self.map["uav_set"]:
          all_uav_coordinates.extend(uav.current_location[:self.coord_count]) 

        user_map = np.zeros((env_dim[0],env_dim[1]))
        for user in self.map["ue_set"]:
            row = user.current_location[0]
            col = user.current_location[1]
            user_map[row][col] = 1
            
        return [self.reshape(np.array(all_uav_coordinates + [self.step_count-self.initial_time])),self.reshape(user_map)]
    

class UserScalableEnvWithTimeInfowithComm(UavSimuEnvWithCommSameReplayIndexesReversedReplay,UserScalableEnvWithTimeInfo):
    
    def get_input_size(self,uav_count):
        return UserScalableEnvWithTimeInfo.get_input_size(self,uav_count)
    
    def get_state_input(self,uav_index):
        return UserScalableEnvWithTimeInfo.get_state_input(self,uav_index)
    
    def create_random_user_locations(self):
        return UserScalableEnvWithTimeInfo.create_random_user_locations(self)
    
    def reset(self):
        UserScalableEnvWithTimeInfo.reset(self)
    
    def reshape(self, data):
        return UserScalableEnvWithTimeInfo.reshape(self,data)
    

In [ ]:
class UserScalableEnvWithTimeInfowithCommFix(UserScalableEnvWithTimeInfowithComm):

    def get_state_input(self,uav_index):
        all_uav_coordinates = []

        for uav in self.map["uav_set"]:
          all_uav_coordinates.extend(uav.current_location[:self.coord_count]) 

        user_map = np.zeros((env_dim[0],env_dim[1]))
        for user in self.map["ue_set"]:
            row = user.current_location[0]%self.env_dim[0]
            col = user.current_location[1]%self.env_dim[1]
            user_map[row][col] = 1
            
        return [self.reshape(np.array(all_uav_coordinates + [self.initial_time%20])),self.reshape(user_map)]
    
    def step(self,uav_index,action_index):
        
        #Get current and new locations of uav
        current_loc = self.getUAV(uav_index).current_location
        new_location = [current_loc[i] + self.actions[action_index][i]
                                                         for i in range(3)]
        
        #Calculate Reward
        reward = 0
        done = False
        if(not self.isInside(new_location) or self.isCollides(uav_index,new_location)):
            reward = PENALTY
            done = True
        else:
            old_sum = self.calculate_sum_rate()
            self.getUAV(uav_index).current_location = new_location
            new_sum = self.calculate_sum_rate()
            reward = new_sum - old_sum
        
        #Get next uav's index
        next_uav_index = (uav_index+1)%(self.uav_count)
        
        #If uav is the last one then increase step
        if(uav_index==self.uav_count-1):
            self.initial_time+=1
            
            if(self.initial_time%UAV_MOVEMENT_SCALE==0):
                self.step_UEs()
            
        #Get next state (Next uav's next input = next observation of next agent)
        next_obs = self.get_state_input(next_uav_index)
        done = done or self.initial_time == self.step_count
        return reward,next_obs,done 

# Log Function

In [ ]:
import pickle
import json
import os

def get_name(*args,**kwargs):
    return "{}_{}_{}LR_{}batch__step{}_iter{}_epsilondecay{}_epsilonmin{}_gamma{}".format(kwargs["agent_class"],
                                                                     kwargs["env"],
                                                                     kwargs["lr"],
                                                                     kwargs["batch_size"],
                                                                     kwargs["step_count"],
                                                                     kwargs["iteration_count"],
                                                                     kwargs["epsilon_decay"],
                                                                     kwargs["epsilon_min"],
                                                                     kwargs["gamma"])
def writeLog(*args, **kwargs):
    
    name = get_name(*args,**kwargs)
    with open("Logs/"+name+".json", 'w') as f:
        json.dump(kwargs, f)
        f.write(os.linesep)
    return name

# Determine Initial UAV locations

In [ ]:
import pickle

env_dim = (100,100)
STEP_COUNT= 64

alt = ALTITUDE
mid_height, mid_width = env_dim[1]/2, env_dim[0]/2


uav_paths = [ 
              [mid_width, mid_height,alt],  \
              [mid_width, mid_height,alt-1], \
              [mid_width, mid_height,alt-2], 
            ]
ue_set = []

UNIT = 20
tmp = UavSimuEnv(uav_paths,ue_set,env_dim)
tmp.render() 

# Parameter Tuning Loop

In [ ]:
import itertools

EPSILON_DECAY = 0.962

learning_rates = [1e-6]
agents = [semiConvAgentCooperative]
Envs=[UserScalableEnvWithTimeInfowithCommFix]
ITERATION_COUNT = 100
STEP_COUNT= 64
COLLECT_STEP_COUNT = STEP_COUNT
batch_sizes = [10]
UAV_MOVEMENT_SCALE = 20
UNIT =20
PENALTY = -100
EPSILON_MIN = 0.1
GAMMA=0.95  # Gamma daha da azaltırsa daha iyi sonuçlar çıkabilir

combinations = itertools.product(learning_rates,Envs,                              
                              batch_sizes,agents)  
for comb in combinations:
    print(comb) 
    LR = comb[0] 
    env = comb[1]
    batch_size = comb[2]
    agent = comb[3]
    
    name = get_name(lr=LR,env=env.__name__,env_dim=env_dim, 
            step_count=STEP_COUNT, iteration_count =ITERATION_COUNT, 
            uav_movement_scale=UAV_MOVEMENT_SCALE, batch_size=batch_size,
            agent_class=agent.__name__, collect_step_size=COLLECT_STEP_COUNT, 
            epsilon_decay=EPSILON_DECAY, epsilon_min=EPSILON_MIN,
            gamma=GAMMA)
     
    environment = env(uav_paths, ue_set, 
                        env_dim, batch_size=batch_size, uav_class=agent)

    per_iter,per_step = simulate(environment,iteration_count=ITERATION_COUNT, 
                              step_count=STEP_COUNT, batch_size = batch_size, 
                              collect_step_size=COLLECT_STEP_COUNT, env_name = name)

    environment.save(name+"_endofsim")

    writeLog(lr=LR, env=env.__name__, per_iter=per_iter, per_step=per_step, env_dim=env_dim, 
          step_count=STEP_COUNT, iteration_count =ITERATION_COUNT, uav_movement_scale=UAV_MOVEMENT_SCALE,
          LastMap = environment.get3DMap(), batch_size=batch_size, agent_class=agent.__name__,
            collect_step_size=COLLECT_STEP_COUNT, epsilon_decay=EPSILON_DECAY, epsilon_min=EPSILON_MIN, gamma=GAMMA) 